# I2C and GPIO Functions

The `mbio` library provides wrapper functions for interacting with GPIO and I2C devices independent of the underlying BSP. The number of each type of device can be found with the `*_num_devices()` function. Devices can then be opened with on of the `*_open` functions to return a handle. It is up to the BSP author to determine which device ID correspondes to the device they want to use. For the Arduino and PMOD IOPs this is abstracted by the `iop_switch` API which returns a handle for a given pin or connector.

With a handle the rest of the `mbio` can be used to read and write from the GPIO pin or I2C peripheral. The complete API for the GPIO and I2C interfaces is:

```C
// mbio interface
int mb_gpio_num_devices();
mb_gpio mb_gpio_open_pin(int index, int channel, int pin);
mb_gpio mb_gpio_open_range(int index, int channel, int low, int high);
mb_gpio mb_gpio_open_all(int index, int channel);
void mb_gpio_set_direction(mb_gpio dev, int direction);
void mb_gpio_write(mb_gpio dev, int val);
int mb_gpio_read(mb_gpio dev);

int mb_i2c_num_devices();
mb_i2c mb_i2c_open(int index);
int mb_i2c_read(mb_i2c device, unsigned char address, char* data, int length);
int mb_i2c_write(mb_i2c device, unsigned char address, const char* data, int length);

// iop_switch interface
mb_i2c iop_switch_i2c_raw(unsigned char scl, unsigned char sda);
mb_i2c iop_switch_i2c_grove(unsigned char port);
mb_gpio iop_switch_gpio_raw(unsigned char pin);
mb_gpio iop_switch_gpio_grove(unsigned char port, unsigned char wire);
```



## Using the GPIO

The Grove connector has two GPIO pins which can be accessed using channels 0 and 1 in the library. First we need to load the overlay and then we can read the value of the two pins back to python. The GPIO library requires creating instances for each pin so two instances are required to access both data pins in a Grove connector.

In [1]:
import ipython_microblaze as ipmb
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

In [2]:
%%microblaze_functions base.PMODA
#include <iop.h>

int read_data(unsigned char* data) {
    gpio g1a = gpio_open_iop_grove(G1, 0);
    gpio g1b = gpio_open_iop_grove(G1, 1);
    gpio_set_direction(g1a, GPIO_IN);
    gpio_set_direction(g1b, GPIO_IN);
    data[0] = gpio_read(g1a);
    data[1] = gpio_read(g1b);
    return 0;
}

In [3]:
data = bytearray(2)
read_data(data)
print(data)

bytearray(b'\x01\x01')


Likewise the same can be done for an Arduino-attached Grove device

In [4]:
%%microblaze_functions base.ARDUINO
#include <iop.h>

int read_data_a(unsigned char* data) {
    gpio g1a = gpio_open_iop_grove(G1, 0);
    gpio g1b = gpio_open_iop_grove(G1, 1);
    gpio_set_direction(g1a, GPIO_IN);
    gpio_set_direction(g1b, GPIO_IN);
    data[0] = gpio_read(g1a);
    data[1] = gpio_read(g1b);
    return 0;
}

In [5]:
read_data_a(data)
print(data)

bytearray(b'\x01\x00')


In [6]:
read_data.reset()
read_data_a.reset()

## Using I2C

The I2C peripherals can be written and read from as per the data sheet. This example communicates the Grove ADC and returns the raw value back to python

In [7]:
%%microblaze_functions base.PMODA
#include <iop.h>
#define ADDRESS 0x50

int adc_read() {
    i2c bus = i2c_open_iop_grove(G4);
    unsigned char data[2];
    // Setup the device
    data[0] = 0x02;
    data[1] = 0x20;
    i2c_write(bus, ADDRESS, data, 2);
    // Read the value
    data[0] = 0x00;
    i2c_write(bus, ADDRESS, data, 1);
    i2c_read(bus, ADDRESS, data, 2);
    return ((int)data[0] << 8) | data[1];
}

In [8]:
adc_read()

78

Likewise we can do the same for Arduino

In [9]:
%%microblaze_functions base.ARDUINO
#include <iop.h>
#define ADDRESS 0x50

int adc_read_a() {
    i2c bus = i2c_open_iop_grove(G4);
    unsigned char data[2];
    // Setup the device
    data[0] = 0x02;
    data[1] = 0x20;
    i2c_write(bus, ADDRESS, data, 2);
    // Read the value
    data[0] = 0x00;
    i2c_write(bus, ADDRESS, data, 1);
    i2c_read(bus, ADDRESS, data, 2);
    return ((int)data[0] << 8) | data[1];
}

In [10]:
adc_read_a()

2175

Note that this is a common pattern for accessing registers in I2C devices - a single byte address write followed by a read or write of one or more bytes.